# ISD-Lite

Download near surface (10 m) wind speed data from the Integrated Surface Dataset (ISD) Lite. We use the [PyISD](https://github.com/CyrilJl/isd-fetch) package. We download `winddirection` [degrees] and `windspeed` [m/s] for 1979-01-01 till 2024-12-31.

In [ ]:
import sys
import os
sys.path.append('../../')
from setup import CWD, SCRATCH_DIR, DATA_DIR
print(CWD)
print(DATA_DIR)
print(SCRATCH_DIR)

In [ ]:
from pyisd import IsdLite
import polars as pl

# Initialize the client
isd = IsdLite(crs=4326, verbose=True)

# View available stations
isd.raw_metadata.sample(5)

In [ ]:
# xmin, ymin, xmax, ymax
geometry = (-105, 25, -90, 50)  # CNA region

In [ ]:
# Query the data -> satellite era, want complete years
cna_data = isd.get_data(
    start='1979-01-01',
    end='2024-12-31',
    geometry=geometry,
    organize_by='field'
)

## Create dataframes

In [ ]:
windspeed = pl.from_pandas(cna_data['windspeed'].reset_index(drop=False))\
    .unpivot(
        index='index',
        variable_name='station',
        value_name='windspeed'
    ).rename({'index': 'time', 'station': 'USAF'})
winddirection = pl.from_pandas(cna_data['winddirection'].reset_index(drop=False))\
    .unpivot(
        index='index',
        variable_name='station',
        value_name='winddirection'
    ).rename({'index': 'time', 'station': 'USAF'})
wind_df = windspeed.join(winddirection, how='left', on=['USAF', 'time']).drop_nulls() # Remove rows with no data
wind_df

## Filter metadata to CNA stations

A station can close then reopen so the same USAF number can have multiple listings in the metadata, this is okay we just need to keep it in mind in the future.

In [ ]:
raw_metadata = pl.from_pandas(isd.raw_metadata[['USAF', 'WBAN', 'STATION NAME', 'ST', 'CALL', 'ELEV(M)', 'BEGIN', 'END', 'x', 'y']])\
    .rename({
        'x': 'lon',
        'y': 'lat',
    })
metadata = raw_metadata.filter(pl.col('USAF').is_in(wind_df['USAF'].unique()))
metadata

## Save dataframes

Whole ISD lite is 1.8 GB once null values are removed

In [ ]:
isd_lite_metadata = os.path.join(DATA_DIR, 'isd_lite_cna_metadata.csv')
isd_lite_file = os.path.join(SCRATCH_DIR, 'isd_lite_cna.csv')

metadata.write_csv(isd_lite_metadata)
wind_df.write_csv(isd_lite_file)